In [ ]:
!pip install numpy PyOpenGL pygame

import pygame
import numpy as np
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import math
import random
import time


# Constants
speed = 0.1  # Camera movement speed
sensitivity = 0.2  # Mouse sensitivity
jump_strength = 0.3  # Strength of the jump
gravity = 0.01  # Gravitational pull on the player
bounce_stop = 0.07
camera_height = 1.75  # Y position of the ground
map_size = 10
walls_height= 10
object_spawn_delay = 1
object_spawn_speed = 1
object_min_radius = 0.25
object_max_radius = 0.5
max_objects_count = 5
max_angle = 1.0
max_link_objects = 50
distance_between_objects = 1.0
vertices = [
    [1, 0, -1],
    [1, 2, -1],
    [-1, 2, -1],
    [-1, 0, -1],
    [1, 0, 1],
    [1, 2, 1],
    [-1, 2, 1],
    [-1, 0, 1]
]
edges = [
    (0, 1), (1, 2), (2, 3), (3, 0),  # Back face edges
    (4, 5), (5, 6), (6, 7), (7, 4),  # Front face edges
    (0, 4), (1, 5), (2, 6), (3, 7)   # Connecting edges
]

def get_rainbow():
    r = random.uniform(0, 1)
    g = random.uniform(0, 1)
    b = random.uniform(0, 1)
    return (r, g, b)

class Plane:
    def __init__(self, position, normal):
        self.position = np.array(position, dtype=float)  # A point on the plane
        self.normal = np.array(normal, dtype=float) / np.linalg.norm(normal)  # Normal vector to the plane

# First-person camera class
class Camera:
    def __init__(self):
        self.position = [0.0, 1.75, 5.0]  # Camera starting position
        self.yaw = 0.0  # Horizontal rotation (yaw)
        self.pitch = 0.0  # Vertical rotation (pitch)
        self.velocity_y = 0.0  # Vertical velocity for jumping/falling
        self.is_jumping = False  # Flag to check if the player is currently jumping

    # Update the camera's yaw and pitch based on mouse movement
    def rotate(self, mouse_dx, mouse_dy):
        self.yaw += mouse_dx * sensitivity
        self.pitch += mouse_dy * sensitivity
        self.pitch = max(-89.0, min(89.0, self.pitch))  # Limit pitch to prevent flipping

    # Move the camera forward, backward, left, right
    def move(self, direction):
        right = [
            math.cos(math.radians(self.yaw)),
            0.0,
            math.sin(math.radians(self.yaw))
        ]
        forward = [
            math.cos(math.radians(self.yaw + 90)),
            0.0,
            math.sin(math.radians(self.yaw + 90))
        ]

        if direction == 'FORWARD':
            self.position[0] -= forward[0] * speed
            self.position[2] -= forward[2] * speed
        elif direction == 'BACKWARD':
            self.position[0] += forward[0] * speed
            self.position[2] += forward[2] * speed
        elif direction == 'LEFT':
            self.position[0] -= right[0] * speed
            self.position[2] -= right[2] * speed
        elif direction == 'RIGHT':
            self.position[0] += right[0] * speed
            self.position[2] += right[2] * speed

    # Apply the camera transformation (position and rotation)
    def apply(self):
        # Apply pitch (look up/down) by rotating around the X-axis
        glRotatef(self.pitch, 1.0, 0.0, 0.0)
        # Apply yaw (look left/right) by rotating around the Y-axis
        glRotatef(self.yaw, 0.0, 1.0, 0.0)
        # Move the camera to the new position
        glTranslatef(-self.position[0], -self.position[1], -self.position[2])

    def update_jump(self):
        # If the player is in the air, apply gravity
        if self.position[1] > camera_height or self.is_jumping:
            self.velocity_y -= gravity  # Gravity pulls down

        # Update the camera's Y position based on velocity
        self.position[1] += self.velocity_y

        # Prevent the player from falling below the ground
        if self.position[1] < camera_height:
            self.position[1] = camera_height
            self.velocity_y = 0.0  # Reset velocity when on the ground
            self.is_jumping = False  # Reset the jump flag

    # Start jumping if the player is on the ground
    def jump(self):
        if self.position[1] == camera_height:
            self.velocity_y = jump_strength  # Set the initial jump velocity
            self.is_jumping = True  # Mark the player as jumping

def draw_cube():
    glBegin(GL_LINES)
    glColor3f(0.7, 0.7, 0.7)  # Light grey color for the grid
    for edge in edges:
        for vertex in edge:
            (vertices[vertex])
    glEnd()

def draw_plane():
    # Draw a 3D plane as the ground
    glBegin(GL_QUADS)
    glColor3f(0.3, 0.6, 0.3)  # Green color for ground
    for x in range(-map_size, map_size):
        for y in range(-map_size, map_size):
            glVertex3f(x, 0, y)  # Ground level Z = 0
            glVertex3f(x + 1, 0, y)
            glVertex3f(x + 1, 0, y + 1)
            glVertex3f(x, 0, y + 1)
    glEnd()

    # Draw grid lines on top of the plane
    glColor3f(0.7, 0.7, 0.7)  # Light grey color for the grid
    glBegin(GL_LINES)
    
    # Draw vertical lines (X-axis)
    for x in range(-map_size, map_size + 1):
        glVertex3f(x, 0, -map_size)
        glVertex3f(x, 0, map_size)
    
    # Draw horizontal lines (Y-axis)
    for y in range(-map_size, map_size + 1):
        glVertex3f(-map_size, 0, y)
        glVertex3f(map_size, 0, y)
    
    glEnd()

def draw_walls():
    # Draw a 3D plane as the left
    glBegin(GL_QUADS)
    glColor4f(1.0, 1.0, 1.0, 0.1)
    
    #left walls
    glVertex3f(-map_size, 0, map_size)  # Ground level Z = 0
    glVertex3f(-map_size, walls_height, map_size)
    glVertex3f(-map_size, walls_height, -map_size)
    glVertex3f(-map_size, 0, -map_size)
    glEnd()

    # Draw a 3D plane as the right
    glBegin(GL_QUADS)
    glColor4f(1.0, 1.0, 1.0, 0.1)
    
    #right walls
    glVertex3f(map_size, 0, map_size)  # Ground level Z = 0
    glVertex3f(map_size, walls_height, map_size)
    glVertex3f(map_size, walls_height, -map_size)
    glVertex3f(map_size, 0, -map_size)
    glEnd()

    # Draw a 3D plane as the back
    glBegin(GL_QUADS)
    glColor4f(1.0, 1.0, 1.0, 0.1)
    
    #back walls
    glVertex3f(map_size, 0, map_size)  # Ground level Z = 0
    glVertex3f(map_size, walls_height, map_size)
    glVertex3f(-map_size, walls_height, map_size)
    glVertex3f(-map_size, 0, map_size)
    glEnd()

    # Draw a 3D plane as the front
    glBegin(GL_QUADS)
    glColor4f(1.0, 1.0, 1.0, 0.1)
    
    #front walls
    glVertex3f(map_size, 0, -map_size)  # Ground level Z = 0
    glVertex3f(map_size, walls_height, -map_size)
    glVertex3f(-map_size, walls_height, -map_size)
    glVertex3f(-map_size, 0, -map_size)
    glEnd()

class VerletObject:
    def __init__(self, position, radius=10.0, color=(255, 255, 255)):
        self.position = np.array(position, dtype=np.float32)
        self.position_last = np.array(position, dtype=np.float32)
        self.acceleration = np.array([0.0, 0.0, 0.0], dtype=np.float32)
        self.radius = radius
        self.color = color

    def update(self, dt):
        # Compute how much we moved
        displacement = self.position - self.position_last
        # Update position
        self.position_last = self.position.copy()
        self.position = self.position + displacement + self.acceleration * (dt ** 2)
        # Reset acceleration
        self.acceleration = np.array([0.0, 0.0, 0.0], dtype=np.float32)

    def accelerate(self, a):
        self.acceleration += np.array(a, dtype=np.float32)

    def set_velocity(self, v, dt):
        self.position_last = self.position - np.array(v, dtype=np.float32) * dt

    def add_velocity(self, v, dt):
        self.position_last -= np.array(v, dtype=np.float32) * dt

    def get_velocity(self, dt):
        return (self.position - self.position_last) / dt

    def draw(self):
        glMatrixMode(GL_MODELVIEW)  # Switch to GL_MODELVIEW mode
        glEnable(GL_COLOR_MATERIAL)
        glPushMatrix()  # Save the current matrix state
        glTranslatef(self.position[0], self.position[1], self.position[2])  # Move to the sphere's position
        quadric = gluNewQuadric()  # Create a new quadric object
        glColor3f(self.color[0], self.color[1], self.color[2])
        gluSphere(quadric, self.radius, 30, 30)  # Draw the sphere (radius, slices, stacks)
        gluDeleteQuadric(quadric)  # Clean up the quadric object
        glPopMatrix()  # Restore the matrix state

class Link:
    def __init__(self, object_1, object_2, target_dist):
        self.object_1 = object_1
        self.object_2 = object_2
        self.target_dist = target_dist

    def apply(self):
        axis = self.object_1.position - self.object_2.position
        dist = np.linalg.norm(axis)
        if dist == 0:
            return  # Avoid division by zero
        n = axis / dist
        delta = self.target_dist - dist
        self.object_1.position += 0.5 * delta * n
        self.object_2.position -= 0.5 * delta * n

class Solver:
    def __init__(self):
        self.objects = []
        self.gravity = np.array([0.0, -10.0, 0], dtype=np.float32)
        self.constraint_min = np.array([-10.0, 0.0, -10.0], dtype=np.float32)
        self.constraint_max = np.array([10.0, 10.0, 10.0], dtype=np.float32)
        self.sub_steps = 1
        self.frame_dt = 0.0
        self.time = 0.0
        self.bounce_retention = 0.9  # Retention factor for bouncing
     
        self.wall_left = Plane(position=(-map_size, walls_height/2, 0), normal=(1, 0, 0))
        self.wall_right = Plane(position=(map_size, walls_height/2, 0), normal=(-1, 0, 0))
        self.wall_back = Plane(position=(0, walls_height/2, map_size), normal=(0, 0, -1))
        self.wall_front = Plane(position=(0, walls_height/2, -map_size), normal=(0, 0, 1))
        
        self.plane = Plane(position=(0, 0, 0), normal=(0, 1, 0)) 
        self.additional_plane = None

        self.links = []

    def add_object(self, position, radius, color):
        obj = VerletObject(position, radius, color)
        self.objects.append(obj)
        return obj

    def add_link(self, object_1, object_2, target_dist):
            link = Link(object_1, object_2, target_dist)
            self.links.append(link)
            return link
            
    def update(self):
        def draw_plane(plane, map_size):
            # Extract plane information
            position = plane.position
            normal = plane.normal

            a = normal[0]
            b = normal[1]
            c = position[1]

            def solve_linear_equation(a, b, c, known_var='x', known_value=0):
                if b == 0:
                    raise ValueError("Parameter 'b' cannot be zero, as it would lead to division by zero.")

                if known_var == 'x':
                    # Solve for y when x is provided
                    x = known_value
                    y = (b * c - a * x) / b
                    return x, y
                elif known_var == 'y':
                    # Solve for x when y is provided
                    y = known_value
                    x = (b * c - b * y) / a
                    return x, y
                else:
                    raise ValueError("The known_var parameter must be either 'x' or 'y'.")
                
            
            # Create two orthogonal vectors on the plane
            glBegin(GL_QUADS)
            glColor4f(0.3, 0.6, 0.3, 0.1)
            
            #left walls
            x, y = solve_linear_equation(a, b, c, known_var='x', known_value=-map_size)
            glVertex3f(x, y, -map_size)  # Ground level Z = 0
            glVertex3f(x, y, map_size)
            x, y = solve_linear_equation(a, b, c, known_var='x', known_value=map_size)
            glVertex3f(x, y, map_size)
            glVertex3f(x, y, -map_size)  # Ground level Z = 0
            glEnd()
        planes = self.get_constraint()
        plane_count = len(planes)
        if plane_count == 6:
            draw_plane(planes[5], map_size=10)

        self.time += self.frame_dt
        step_dt = self.get_step_dt()
        for _ in range(self.sub_steps):
            self.apply_gravity()
            self.check_collisions(step_dt)
            self.apply_constraint(step_dt)
            self.update_objects(step_dt)
            self.apply_link()

    def set_simulation_update_rate(self, rate):
        self.frame_dt = 1.0 / rate

    def set_constraint(self, min_position, max_position):
        self.constraint_min = np.array(min_position, dtype=np.float32)
        self.constraint_max = np.array(max_position, dtype=np.float32)

    def set_sub_steps_count(self, sub_steps):
        self.sub_steps = sub_steps

    def set_object_velocity(self, obj, v):
        obj.set_velocity(v, self.get_step_dt())

    def get_objects(self):
        return self.objects

    def get_constraint(self):
        if self.additional_plane != None:
            return [self.plane, self.wall_back, self.wall_front, self.wall_left, self.wall_right, self.additional_plane]
        return [self.plane, self.wall_back, self.wall_front, self.wall_left, self.wall_right]

    def get_objects_count(self):
        return len(self.objects)

    def get_time(self):
        return self.time

    def get_step_dt(self):
        return self.frame_dt / self.sub_steps

    def apply_gravity(self):
        for obj in self.objects:
            obj.accelerate(self.gravity)

    def check_collisions(self, dt):
        response_coef = 0.75
        objects_count = len(self.objects)
        for i in range(objects_count):
            object_1 = self.objects[i]
            for k in range(i + 1, objects_count):
                object_2 = self.objects[k]
                v = object_1.position - object_2.position
                dist2 = np.dot(v, v)
                min_dist = object_1.radius + object_2.radius
                if dist2 < min_dist * min_dist:
                    dist = np.sqrt(dist2)
                    n = v / dist
                    mass_ratio_1 = object_1.radius / (object_1.radius + object_2.radius)
                    mass_ratio_2 = object_2.radius / (object_1.radius + object_2.radius)
                    delta = 0.5 * response_coef * (dist - min_dist)
                    object_1.position -= n * (mass_ratio_2 * delta)
                    object_2.position += n * (mass_ratio_1 * delta)

    def apply_constraint(self, dt): 
        response_coef = 0.75
        objects_count = len(self.objects)
        planes = self.get_constraint()
        plane_count = len(planes)
        for i in range(objects_count):
            object_1 = self.objects[i]
            for k in range(plane_count):
                plane = planes[k]
                v = object_1.position - plane.position
                dist_to_plane = np.dot(v, plane.normal)  # Project vector onto plane normal to get distance
                if dist_to_plane < object_1.radius:
                    # Handle collision response with plane (position adjustment)
                    penetration_depth = object_1.radius - dist_to_plane
                    object_1.position += plane.normal * (response_coef * penetration_depth)

                    # Split velocity into normal and tangential components
                    velocity = object_1.get_velocity(dt)  # Assuming object_1 has a velocity attribute

                    # Normal component of velocity
                    v_n = np.dot(velocity, plane.normal) * plane.normal

                    # Tangential component of velocity
                    v_t = velocity - v_n

                    # Update normal component using coefficient of restitution
                    e = response_coef  # Using response coefficient as restitution coefficient
                    v_n_new = -e * v_n

                    # New velocity after collision
                    object_1.set_velocity(v_n_new + v_t, dt)
    
    def apply_link(self):
            for link in self.links:
                link.apply()
                
    def update_objects(self, dt):
        for obj in self.objects:
            obj.update(dt)
    
    def remove_objects(self):
        self.objects = []

    def update_plane(self):
        self.additional_plane = Plane(position=(0, 2, 0), normal=(random.uniform(0, 0.5), 1, 0))
            

def main():
    pygame.init()
    display = (800, 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL)
    clock = pygame.time.Clock()

    # glEnable(GL_DEPTH_TEST)
    glEnable(GL_BLEND)
    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, (display[0] / display[1]), 0.1, 100.0)  # Perspective projection

    # Switch to ModelView Matrix (for object transformations)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    camera = Camera()
    
    pygame.mouse.set_visible(False)  # Hide the mouse for first-person control
    pygame.mouse.set_pos(display[0] // 2, display[1] // 2)  # Center mouse
    pygame.event.set_grab(True)  # Capture mouse input for smooth camera movement

    solver = Solver()
    solver.set_simulation_update_rate(60)
    solver.set_sub_steps_count(4)
    objects = []
    link_objects = []
    spawn_timer = time.time()

    # Main loop
    while True:
        dt = clock.tick(60) / 1000.0
        solver.frame_dt = dt

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
        
        keys = pygame.key.get_pressed()
        if keys[K_w]:
            camera.move('FORWARD')
        if keys[K_s]:
            camera.move('BACKWARD')
        if keys[K_a]:
            camera.move('LEFT')
        if keys[K_d]:
            camera.move('RIGHT')

        if keys[K_SPACE]:
            camera.jump()
        if keys[K_r]:
            objects = []
            solver.remove_objects()
            link_objects = []
        if keys[K_t]:
            solver.update_plane()

        # Get mouse movement for camera rotation
        mouse_dx, mouse_dy = pygame.mouse.get_rel()
        camera.rotate(mouse_dx, mouse_dy)
        camera.update_jump()

        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

        # Apply the camera transformations (first-person view)
        glLoadIdentity()
        camera.apply()

        if len(objects) < max_objects_count and (time.time() - spawn_timer) >= object_spawn_delay:
            spawn_timer = time.time()
            radius = random.uniform(object_min_radius, object_max_radius)
            position = (0, 10, 0)
            t = pygame.time.get_ticks() / 1000  # time in seconds
            obj = solver.add_object(position, radius, get_rainbow())
            angle = max_angle * math.sin(t) + math.pi * 0.5
            obj.set_velocity(object_spawn_speed * pygame.math.Vector3(math.cos(angle), math.sin(angle) ,math.sin(angle)), dt)
            objects.append(obj)

        # Create a list of Verlet objects to form a rope
        distance_between_objects = 0.05
        if len(link_objects) < max_link_objects:
            link_objects = [solver.add_object([i * distance_between_objects, 0 + i * distance_between_objects, i * distance_between_objects], distance_between_objects, get_rainbow()) for i in range(max_link_objects)]
            
            for i in range(max_link_objects - 1):
                solver.add_link(link_objects[i], link_objects[i + 1], distance_between_objects + distance_between_objects)

        solver.update()
        draw_plane()
        draw_walls()
        draw_cube()
        for obj in solver.get_objects():
            obj.draw()
        pygame.display.flip()
        pygame.time.wait(10)

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: video system not initialized

: 